# Rheological Protocol Validation Index

**Dashboard for tracking protocol validation across all rheological experiments.**

This notebook provides:
1. Protocol coverage matrix (which data files support which protocols)
2. Links to individual protocol validation notebooks
3. Summary of validation status
4. Instructions for running the full validation suite

## Supported Protocols

| Protocol | Description | Key Checks |
|----------|-------------|------------|
| **Flow Curve** | σ vs γ̇ at steady state | Shear thinning, positive values |
| **Creep** | J(t) = γ(t)/σ₀ | Monotonic compliance |
| **Stress Relaxation** | G(t) = σ(t)/γ₀ | Monotonic decay |
| **Startup Shear** | σ(t) at constant γ̇ | Overshoot detection |
| **SAOS** | G'(ω), G''(ω) | Positive moduli, KK consistency |
| **LAOS** | σ(γ, t) nonlinear | Harmonic analysis, Lissajous closure |

In [ ]:
# Configuration
MODE = "FAST"  # "FAST" or "FULL"

if MODE == "FAST":
    MAX_FILES_PER_PROTOCOL = 2
    SKIP_HEAVY_PLOTS = True
    SAVE_ARTIFACTS = False
else:
    MAX_FILES_PER_PROTOCOL = None  # all files
    SKIP_HEAVY_PLOTS = False
    SAVE_ARTIFACTS = True

print(f"Running in {MODE} mode")

In [ ]:
from pathlib import Path
import sys

# Add project root to path for imports
project_root = Path.cwd()
while not (project_root / "pyproject.toml").exists() and project_root != project_root.parent:
    project_root = project_root.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from examples.verification.utils.validation_utils import (
    discover_files_by_protocol,
    get_data_dir,
    Protocol,
)

print(f"Project root: {project_root}")
print(f"Data directory: {get_data_dir()}")

## 1. Data Inventory

Discover available data files for each protocol.

In [ ]:
import pandas as pd

protocols: list[Protocol] = [
    "flow_curve",
    "creep",
    "stress_relaxation",
    "startup_shear",
    "saos",
    "laos",
]

data_dir = get_data_dir()

# Build inventory
inventory = []
for protocol in protocols:
    files = discover_files_by_protocol(data_dir, protocol)
    inventory.append({
        "Protocol": protocol,
        "File Count": len(files),
        "Extensions": ", ".join(sorted(set(f.suffix for f in files))),
        "Directories": ", ".join(sorted(set(f.parent.name for f in files))),
    })

inventory_df = pd.DataFrame(inventory)
inventory_df

## 2. Protocol Coverage Matrix

Sample files for each protocol (first 3 per protocol).

In [ ]:
for protocol in protocols:
    files = discover_files_by_protocol(data_dir, protocol)
    print(f"\n{'='*60}")
    print(f"Protocol: {protocol.upper()} ({len(files)} files)")
    print(f"{'='*60}")
    for f in files[:3]:
        print(f"  - {f.relative_to(data_dir)}")
    if len(files) > 3:
        print(f"  ... and {len(files) - 3} more")

## 3. Validation Notebook Links

Click to open each protocol validation notebook:

| Notebook | Protocol | Status |
|----------|----------|--------|
| [01_validate_flow_curve.ipynb](01_validate_flow_curve.ipynb) | Flow Curve | Ready |
| [02_validate_creep.ipynb](02_validate_creep.ipynb) | Creep | Ready |
| [03_validate_stress_relaxation.ipynb](03_validate_stress_relaxation.ipynb) | Stress Relaxation | Ready |
| [04_validate_startup_shear.ipynb](04_validate_startup_shear.ipynb) | Startup Shear | Ready |
| [05_validate_saos.ipynb](05_validate_saos.ipynb) | SAOS | Ready |
| [06_validate_laos.ipynb](06_validate_laos.ipynb) | LAOS | Ready |

## 4. Validation Status Summary

Check for existing validation reports from previous runs.

In [ ]:
import json
from examples.verification.utils.validation_utils import get_output_dir

status_summary = []

for protocol in protocols:
    output_dir = get_output_dir(protocol)
    report_path = output_dir / "validation_report.json"
    
    if report_path.exists():
        with open(report_path) as f:
            report = json.load(f)
        status = "PASS" if report.get("all_passed", False) else "FAIL"
        n_files = report.get("n_files_validated", "?")
        timestamp = report.get("generated_at", "unknown")[:19]
    else:
        status = "NOT RUN"
        n_files = "-"
        timestamp = "-"
    
    status_summary.append({
        "Protocol": protocol,
        "Status": status,
        "Files Validated": n_files,
        "Last Run": timestamp,
    })

status_df = pd.DataFrame(status_summary)
status_df

## 5. Instructions for Running Validation Suite

### Quick Validation (FAST mode)

```bash
# Run all validation notebooks in FAST mode (2 files per protocol)
cd examples/verification
for nb in 0[1-6]_validate_*.ipynb; do
    uv run jupyter execute "$nb" --inplace
done
```

### Full Validation (FULL mode)

Edit each notebook to set `MODE = "FULL"`, then run:

```bash
# Run all validation notebooks in FULL mode (all files)
cd examples/verification
for nb in 0[1-6]_validate_*.ipynb; do
    uv run jupyter execute "$nb" --inplace
done
```

### Individual Protocol

```bash
# Run a single protocol validation
uv run jupyter execute examples/verification/01_validate_flow_curve.ipynb --inplace
```

## 6. Output Directory Structure

Validation artifacts are saved to:

```
examples/verification/outputs/
├── flow_curve/
│   ├── cleaned_data/      # Preprocessed data files
│   ├── derived_quantities/ # Computed quantities (η, J, G, etc.)
│   ├── plots/             # Validation plots
│   └── validation_report.json
├── creep/
├── stress_relaxation/
├── startup_shear/
├── saos/
└── laos/
```

In [ ]:
# Check output directory structure
outputs_root = Path("outputs")
if outputs_root.exists():
    for protocol_dir in sorted(outputs_root.iterdir()):
        if protocol_dir.is_dir():
            n_files = sum(1 for _ in protocol_dir.rglob("*") if _.is_file())
            print(f"{protocol_dir.name}/: {n_files} files")
else:
    print("Output directories not yet created. Run validation notebooks first.")

## References

Standard rheological protocol references:

- **SAOS**: Ferry, J.D. "Viscoelastic Properties of Polymers" (1980)
- **LAOS**: Ewoldt et al. "New measures for characterizing nonlinear viscoelasticity" (2008) J. Rheol.
- **Flow curves**: Macosko, C.W. "Rheology: Principles, Measurements, and Applications" (1994)
- **Creep/Relaxation**: Tschoegl, N.W. "Phenomenological Theory of Linear Viscoelastic Behavior" (1989)